In [2]:
from data_funcs import get_data
import numpy as np
import pandas as pd

port, df = get_data()
df['stocks']['AMD']

100%|██████████| 30/30 [00:03<00:00,  8.70it/s]


,Adj Close,Close,High,Low,Open,Volume,day sin,returns
0,178.279999,178.279999,179.869995,178.229996,179.555695,2173613.0,-0.625923,-0.007129
1,178.281998,178.281998,178.779999,178.149994,178.289993,1472620.0,-0.642788,0.000011
2,177.024994,177.024994,178.360001,177.009995,178.300003,2444764.0,-0.659346,-0.007051
3,175.880005,175.880005,177.044998,175.750000,177.024994,2305916.0,-0.675590,-0.006468
4,176.759995,176.759995,177.179993,175.850006,175.899994,1635446.0,-0.691513,0.005003
...,...,...,...,...,...,...,...,...
2950,180.242004,180.242004,180.479996,180.100006,180.240005,565454.0,-0.923880,0.000111
2951,180.059998,180.059998,180.379898,179.882401,180.210007,550152.0,-0.915311,-0.001010
2952,180.179901,180.179901,180.236893,179.919998,180.044998,565001.0,-0.906308,0.000666
2953,179.985001,179.985001,180.199905,179.940002,180.179993,449592.0,-0.896873,-0.001082


In [ ]:
def calc_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = df['High'] - df['Close'].shift(1)
    low_close = df['Low'] - df['Close'].shift(1)
    
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1, skipna=False)
    atr = true_range.ewm(com=period, min_periods=period).mean()
    return atr

calc_atr(df['stocks']['NVDA'])

2024-01-08 12:50:00-05:00         NaN
2024-01-08 12:55:00-05:00         NaN
2024-01-08 13:00:00-05:00         NaN
2024-01-08 13:05:00-05:00         NaN
2024-01-08 13:10:00-05:00         NaN
                               ...   
2024-03-01 15:35:00-05:00    1.519538
2024-03-01 15:40:00-05:00    1.510903
2024-03-01 15:45:00-05:00    1.462178
2024-03-01 15:50:00-05:00    1.449367
2024-03-01 15:55:00-05:00    1.530209
Length: 2903, dtype: float64

In [ ]:
def calc_rsi(df, period=14):
    """
    Calculate the Relative Strength Index (RSI)
    """
    delta = df['Close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    average_gain = up.rolling(window=period).mean()
    average_loss = abs(down.rolling(window=period).mean())

    rs = average_gain / average_loss
    rsi = (100 - (100 / (1 + rs)))/100
    
    return rsi

calc_rsi(df['efts']['VOO'])

2024-01-08 12:50:00-05:00         NaN
2024-01-08 12:55:00-05:00         NaN
2024-01-08 13:00:00-05:00         NaN
2024-01-08 13:05:00-05:00         NaN
2024-01-08 13:10:00-05:00         NaN
                               ...   
2024-03-01 15:35:00-05:00    0.386408
2024-03-01 15:40:00-05:00    0.496840
2024-03-01 15:45:00-05:00    0.572320
2024-03-01 15:50:00-05:00    0.572523
2024-03-01 15:55:00-05:00    0.519394
Name: Close, Length: 2924, dtype: float64

In [ ]:
import pandas as pd
import numpy as np

def calc_adx(df, period=14):
    """
    Calculate the Average Directional Index (ADX)
    """
    H = df['High'] - df['High'].shift(1)
    L = df['Low'].shift(1) - df['Low']
    
    pdm = np.where((H > L) & (H > 0), H, 0)
    ndm = np.where((H < L) & (L > 0), L, 0)
    
    pdi = (pdm/df['atr']).ewm(com=period, min_periods=period).mean()
    ndi = (ndm/df['atr']).ewm(com=period, min_periods=period).mean()

    adx = (abs(pdi-ndi) / (pdi+ndi)).ewm(com=period, min_periods=period).mean()

    return adx

calc_adx(df['stocks']['NVDA'])

2024-01-08 12:50:00-05:00         NaN
2024-01-08 12:55:00-05:00         NaN
2024-01-08 13:00:00-05:00         NaN
2024-01-08 13:05:00-05:00         NaN
2024-01-08 13:10:00-05:00         NaN
                               ...   
2024-03-01 15:35:00-05:00    0.192548
2024-03-01 15:40:00-05:00    0.190218
2024-03-01 15:45:00-05:00    0.183076
2024-03-01 15:50:00-05:00    0.179573
2024-03-01 15:55:00-05:00    0.174005
Name: atr, Length: 2903, dtype: float64

In [ ]:
def  KPIs(df, rfr=0.02):
    """
    Calculate the Compound Annual Growth Rate (CAGR) and the annualized volatility
    """   
    n = len(df['returns']) / 252/78   # 78 inputs per day (NY Stock exchange is open 6.5 hr per day)
    cum_ret = (1+df['returns']).cumprod()
    cagr = (cum_ret.iloc[-1]) ** (1/n) - 1
    
    vol = df['returns'][df['returns']<rfr].std() * np.sqrt(252*78)
    
    return cagr, vol

KPIs(df['stocks']['NVDA'])

(22.078577321916107, 0.3659440703906821)

In [ ]:
def tckr_Sortino(df, rfr=0.01):
    """
    Calculate the Sortino Ratio
    """
    cagr, neg_vol = KPIs(df)
    return (cagr- rfr) / neg_vol

tckr_Sortino(df['stocks']['NVDA'])

60.30587487961119

## Join all KPIs and Sortino

In [ ]:
measurements = pd.DataFrame()

for eq in port.keys():
    for stck in port[eq]:
        measurements[stck] = [df[eq][stck]['adx'].iloc[-1], df[eq][stck]['rsi'].iloc[-1], tckr_Sortino(df[eq][stck])]
        
measurements = measurements.transpose()
measurements.columns = ['adx', 'rsi', 'sortino']

In [ ]:
measurements

,adx,rsi,sortino
AMD,0.210476,0.615899,17.834805
BYDDY,0.243130,0.294111,-1.180583
ZI,0.203343,0.644445,0.650755
PFE,0.251158,0.431153,-2.367650
NU,0.318551,0.424242,11.389546
WMT,0.283518,0.592061,8.049181
F,0.114153,0.486256,1.260779
GOOGL,0.333639,0.239265,-0.271213
Z,0.145643,0.544158,0.582827
NVDA,0.174005,0.589541,60.305875
